In [1]:
import requests
import grab
import json
import logging
import codecs
from lxml.etree import ElementTree, fromstring
from bs4 import BeautifulSoup
from tqdm import tqdm
from multiprocessing.dummy import Pool as ThreadPool
from IPython.display import HTML

In [ ]:
url = 'http://www.cyberforum.ru/members/list-page1.html?pp=100&order=desc&sort=lastvisit'
headers = {
    'User-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:56.0) Gecko/20100101 Firefox/56.0'
}
cookies = {
    ...
}
g = grab.Grab(headers=headers, cookies=cookies)

USR_ID = []

try:
    g.go(url, timeout=30)

    for x in g.doc.select("//td[@class='alt1Active']//a/@href"):
        print x.text()
        USR_ID.append(x.text())
        
except Exception as e:
    logging.error(str(e))

In [ ]:
USR_ID

In [101]:
def process_page(x):
    results = []
    g = grab.Grab(headers=headers, cookies=cookies)
    try:
        i = 1
        while True:
            g.go('http://www.cyberforum.ru/members/list-page%d.html?pp=100&order=desc&sort=lastvisit' % x, timeout=30)
            members = g.doc.select("//td[@class='alt1Active']//a/@href")
            if not members:
                break
            for x in members:
                results.append(x.text())
            else:
                break
            i += 1
    except Exception as e:
        logging.error('%s %s' % (x, str(e)))
    return results

In [102]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [103]:
%%time
pool_size = 16
pages_amount = 54
PAGES = range(1, pages_amount)
users = []

with codecs.open('./data_mp.csv', 'w', encoding='utf-8') as f_out:
    total_reviews = 0
    for x in tqdm(chunks(PAGES, pool_size), total=len(PAGES) // pool_size):
        pool = ThreadPool(pool_size)
        res = pool.map(process_page, x)
        pool.close()
        pool.join()
        for r in res:
            users.extend(r)

CPU times: user 2.01 s, sys: 144 ms, total: 2.16 s
Wall time: 1min 13s


In [109]:
len(users)

5300

In [112]:
with codecs.open("users.csv", "w", encoding='utf-8') as f:
    for a in users:
        f.write(a)
        f.write('\n')

In [253]:
def process_member(s):
    results = {}
    results['url'] = s
    g = grab.Grab(headers=headers, cookies=cookies)
    try:
        i = 1
        while True:
            g.go(s, timeout=30)
            
            for x in g.doc.select("//td[@id='username_box']//h1"):
                results['name'] = x.text()
            
            last_visit = g.doc.select("//div[@id='last_online']")
            for x in last_visit:
                t = x.text().split()[2]
                t.replace(' ', '')
                results['last_visit'] = t
                if (t == u'Сегодня'):
                    results['last_visit'] = '12.11.2017'
                    continue
                if (t == u'Вчера'):
                    results['last_visit'] = '11.11.2017'
                    continue
                    
            t = [x.text() for x in g.doc.select("//dl[@class='smallfont list_no_decoration profilefield_list']//dd")]
            if (len(t) == 5):
                results['bdate'] = t[0]
                results['rdate'] = t[1]
                t[2] = t[2].replace(',', '')
                results['msg_total'] = t[2]
                results['reputation'] = t[3]
                results['blog_total'] = t[4]
            else:
                results['rdate'] = t[0]
                t[1] = t[1].replace(',', '')
                results['msg_total'] = t[1]
                results['reputation'] = t[2]
                results['blog_total'] = t[3]
            
            for x in g.doc.select("//div[@class='alt2 smallfont block_row block_footer']//strong"):
                results['visits_num'] = x.text().replace(',', '')
                
            lst = [x.text() for x in g.doc.select("//ul[@class='list_no_decoration']//li[@class='profilefield_category']//dl[@class='list_no_decoration profilefield_list']/*[self::dt or self::dd]")]
            for i in range(0, len(lst) / 2):
                results[lst[i*2]] = lst[i*2+1]
                
            for x in g.doc.select("//div[@class='friends_counter']"):
                results['friends_num'] = x.text().split()[3]
            if (not ('friends_num' in results)):
                results['friends_num'] = 0
                
            for x in g.doc.select("//div[@id='stats']//fieldset/ul/li"):
                t = x.text().split(': ')
                if (len(t) < 2):
                    continue
                if (t[0] == u'Последняя запись в блоге'):
                    if (t[1] == u'никогда'):
                        t[1] = 'never'
                    results['blog_last'] = t[1]
                t[1] = t[1].replace(' ', '')
                if (t[0] == u'Получено положительных оценок'):
                    t[1] = t[1].replace(',', '')
                    results['got_positive_marks'] = t[1]
                if (t[0] == u'Получено отрицательных оценок'):
                    t[1] = t[1].replace(',', '')
                    results['got_negative_marks'] = t[1]
                if (t[0] == u'Отдано положительных оценок'):
                    t[1] = t[1].replace(',', '')
                    results['given_positive_marks'] = t[1]
                if (t[0] == u'Отдано отрицательных оценок'):
                    t[1] = t[1].replace(',', '')
                    results['given_negative_marks'] = t[1]
                if (t[0] == u'Рефералы'):
                    t[1] = t[1].replace(',', '')
                    results['referrals'] = t[1]
                if (t[0] == u'Сообщений в день'):
                    t[1] = t[1].replace(',', '')
                    results['msg_per_day'] = t[1]
                    
                lst = [1 for x in g.doc.select("//img[@id='user_avatar']")]
                if (len(lst) > 0):
                    results['avatar'] = 1
                else:
                    results['avatar'] = 0
            
            else:
                break
            i += 1
    except Exception as e:
        logging.error('%s %s' % (x, str(e)))
    return results

In [242]:
url = 'http://www.cyberforum.ru/members/1016422.html'
d = process_member(url)
for k in d.keys():
    print k, ' ', d[k]

bdate   17 August
name   KeLZUQ18Lk
last_visit   14.05.2017
url   http://www.cyberforum.ru/members/1016422.html
visits_num   50
friends_num   0
given_positive_marks   0
referrals   0
rdate   14.05.2017
reputation   0
avatar   0
blog_last   never
given_negative_marks   0
blog_total   0
msg_per_day   0
got_negative_marks   0
got_positive_marks   0
msg_total   0


In [243]:
users[:10]

['http://www.cyberforum.ru/members/590010.html',
 'http://www.cyberforum.ru/members/1100965.html',
 'http://www.cyberforum.ru/members/808361.html',
 'http://www.cyberforum.ru/members/891359.html',
 'http://www.cyberforum.ru/members/590810.html',
 'http://www.cyberforum.ru/members/1111547.html',
 'http://www.cyberforum.ru/members/312486.html',
 'http://www.cyberforum.ru/members/1110747.html',
 'http://www.cyberforum.ru/members/692124.html',
 'http://www.cyberforum.ru/members/1112107.html']

In [258]:
pool_size = 16
dicts = []

with codecs.open('./data_mp.csv', 'w', encoding='utf-8') as f_out:
    total_reviews = 0
    for x in tqdm(chunks(users, pool_size), total=len(users) // pool_size):
        pool = ThreadPool(pool_size)
        res = pool.map(process_member, x)
        pool.close()
        pool.join()
        for r in res:
            dicts.append(r)

In [255]:
for d in dicts:
    for k in d.keys():
        print k, ' ', d[k]

referrals   0
name   medok
last_visit   11.11.2017
url   http://www.cyberforum.ru/members/590010.html
visits_num   4
friends_num   0
msg_total   2
given_positive_marks   0
rdate   20.05.2015
given_negative_marks   0
avatar   0
blog_last   never
blog_total   0
msg_per_day   0.00
got_negative_marks   0
got_positive_marks   0
reputation   0
bdate   26 March 1998 (19)
name   Ustura
last_visit   12.11.2017
url   http://www.cyberforum.ru/members/1100965.html
visits_num   3
friends_num   0
given_positive_marks   18
referrals   0
rdate   25.10.2017
reputation   0
avatar   0
blog_last   never
given_negative_marks   0
blog_total   0
msg_per_day   1.57
got_negative_marks   0
got_positive_marks   0
msg_total   27
bdate   17 February 1996 (21)
name   Diman9902
last_visit   11.11.2017
url   http://www.cyberforum.ru/members/808361.html
visits_num   3
friends_num   0
given_positive_marks   0
referrals   0
rdate   16.04.2016
reputation   0
avatar   0
blog_last   never
given_negative_marks   0
blog_tota

friends_num   0
given_positive_marks   1
referrals   0
rdate   29.10.2017
reputation   0
avatar   0
blog_last   never
given_negative_marks   0
blog_total   0
msg_per_day   0.31
got_negative_marks   0
got_positive_marks   0
msg_total   4
bdate   29 April 2000 (17)
name   Олег Т
last_visit   11.11.2017
url   http://www.cyberforum.ru/members/1110737.html
visits_num   10
friends_num   0
given_positive_marks   2
referrals   0
rdate   09.11.2017
reputation   0
avatar   0
blog_last   never
given_negative_marks   0
blog_total   0
msg_per_day   1.95
got_negative_marks   0
got_positive_marks   0
msg_total   4
referrals   0
name   val-s
last_visit   12.11.2017
url   http://www.cyberforum.ru/members/1045865.html
visits_num   3
friends_num   0
msg_total   0
given_positive_marks   0
rdate   27.06.2017
given_negative_marks   0
avatar   0
blog_last   never
blog_total   0
msg_per_day   0
got_negative_marks   0
got_positive_marks   0
reputation   0


In [249]:
import pandas as pd

In [259]:
df = pd.DataFrame(dicts)

In [260]:
df.set_index(keys='name')

,avatar,bdate,blog_last,blog_total,friends_num,given_negative_marks,given_positive_marks,got_negative_marks,got_positive_marks,last_visit,...,url,visits_num,Интересы,Конфигурация компьютера,Местоположение,О себе,Реальное имя,Специализация,Стаж работы,Чем занимаетесь
name,,,,,,,,,,,,,,,,,,,,,
medok,0.0,NaN,never,0,0,0,0,0,0,11.11.2017,...,http://www.cyberforum.ru/members/590010.html,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ustura,0.0,26 March 1998 (19),never,0,0,0,18,0,0,12.11.2017,...,http://www.cyberforum.ru/members/1100965.html,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Diman9902,0.0,17 February 1996 (21),never,0,0,0,0,0,0,11.11.2017,...,http://www.cyberforum.ru/members/808361.html,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GameMiner,0.0,18 June 2001 (16),never,0,0,0,0,0,0,11.11.2017,...,http://www.cyberforum.ru/members/891359.html,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
evikza,1.0,10 January 1992 (25),never,0,1,0,59,2,210,12.11.2017,...,http://www.cyberforum.ru/members/590810.html,214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vanac1231,0.0,NaN,never,0,0,0,0,0,0,11.11.2017,...,http://www.cyberforum.ru/members/1111547.html,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
droider,1.0,9 June 1986 (31),never,0,19,0,107,0,2229,11.11.2017,...,http://www.cyberforum.ru/members/312486.html,1161,"программирование, IT, математика, арабский язык",NaN,"Чеченская Республика, г. Грозный",Чеченец. Окончил Грозненский Государственный Н...,Руслан,"Android, Pascal, Delphi, Базы данных, Железо, ...",Свыше 10 лет,"разработчик БД, SysAdmin"
Богдан Хакер,0.0,11,never,0,0,0,3,0,1,12.11.2017,...,http://www.cyberforum.ru/members/1110747.html,4,NaN,Windows 7,Россия,NaN,NaN,C++,NaN,NaN
qwert228,0.0,15 January 1991 (26),never,0,0,0,6,0,0,11.11.2017,...,http://www.cyberforum.ru/members/692124.html,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [262]:
df.to_csv('data.csv', encoding='utf-8')